In [0]:
%run ./02config

In [0]:

# COMMAND ----------

# MAGIC %md
# MAGIC ## Define Schema

# COMMAND ----------

# Event payload schema
event_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("customer_name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("payment_method", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("discount_pct", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("base_price", DoubleType(), True),
    StructField("unit_price", DoubleType(), True),
    StructField("order_timestamp", StringType(), True)
])

print("✓ Event schema defined")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stream 1: Read from Event Hub and Parse Events

# COMMAND ----------

print("="*70)
print("STARTING EVENT HUB STREAMING")
print("="*70)

# Read streaming data from Event Hub
raw_stream = (spark.readStream
    .format("kafka")
    .options(**KAFKA_OPTIONS)
    .load()
)

# Parse JSON payload and add metadata
parsed_stream = (raw_stream
    .selectExpr("CAST(value AS STRING) as json_value", "timestamp", "offset", "partition")
    .withColumn("parsed_data", from_json(col("json_value"), event_schema))
    .select(
        col("parsed_data.*"),
        col("timestamp").alias("event_time"),
        col("offset").alias("kafka_offset"),
        col("partition").alias("partition_id")
    )
    .withColumn("bronze_timestamp", current_timestamp())
    .withColumn("order_timestamp_parsed", to_timestamp(col("order_timestamp")))
)

print("✓ Stream parsing configured")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stream 2: Write Orders to Bronze Table

# COMMAND ----------

print("\n" + "="*70)
print("STARTING ORDERS STREAM")
print("="*70)

# Select orders columns
orders_stream = parsed_stream.select(
    col("order_id"),
    col("customer_id"),
    col("customer_name"),
    col("location"),
    col("product_id"),  # FK to products
    col("order_status"),
    col("payment_method"),
    col("quantity"),
    col("discount_pct"),
    col("total_amount"),
    col("order_timestamp"),
    col("order_timestamp_parsed"),
    col("event_time"),
    col("kafka_offset"),
    col("partition_id"),
    col("bronze_timestamp")
)

# Write orders stream to Delta table
orders_query = (orders_stream
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", orders_checkpoint)
    .option("mergeSchema", "true")
   # .trigger(processingTime="10 seconds")  # Micro-batch every 10 seconds
     .trigger(once=True) # for testing
    .toTable(bronze_orders_table)
)

print(f"✓ Orders stream started")
print(f"  Query ID: {orders_query.id}")
print(f"  Target: {bronze_orders_table}")
print(f"  Checkpoint: {orders_checkpoint}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stream 3: Deduplicate and Write Products to Bronze Table

# COMMAND ----------

print("\n" + "="*70)
print("STARTING PRODUCTS STREAM")
print("="*70)

# Select and deduplicate products
products_stream = (parsed_stream
    .select(
        col("product_id"),
        col("product_name"),
        col("category"),
        col("brand"),
        col("base_price"),
        col("unit_price"),
        col("bronze_timestamp")
    )
    .dropDuplicates(["product_id"])  # Keep only unique products per batch
)

# Write products stream to Delta table with merge logic
products_query = (products_stream
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", products_checkpoint)
    .option("mergeSchema", "true")
    #.trigger(processingTime="10 seconds")
    .trigger(once=True) # for testing
    .foreachBatch(lambda batch_df, batch_id: upsert_products(batch_df, batch_id))
    .start()
)

def upsert_products(batch_df, batch_id):
    """
    Upsert products to avoid duplicates across batches
    """
    if batch_df.count() == 0:
        return
    
    # Create temp view
    batch_df.createOrReplaceTempView("products_batch")
    
    # Merge logic
    merge_query = f"""
    MERGE INTO {bronze_products_table} target
    USING products_batch source
    ON target.product_id = source.product_id
    WHEN MATCHED THEN
        UPDATE SET
            target.product_name = source.product_name,
            target.category = source.category,
            target.brand = source.brand,
            target.base_price = source.base_price,
            target.unit_price = source.unit_price,
            target.bronze_timestamp = source.bronze_timestamp
    WHEN NOT MATCHED THEN
        INSERT (product_id, product_name, category, brand, base_price, unit_price, bronze_timestamp)
        VALUES (source.product_id, source.product_name, source.category, source.brand, 
                source.base_price, source.unit_price, source.bronze_timestamp)
    """
    
    # Execute merge
    try:
        spark.sql(merge_query)
        print(f"  Batch {batch_id}: Upserted {batch_df.count()} products")
    except Exception as e:
        # If table doesn't exist, create it
        if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
            batch_df.write.format("delta").mode("append").saveAsTable(bronze_products_table)
            print(f"  Batch {batch_id}: Created table and inserted {batch_df.count()} products")
        else:
            raise e

print(f"✓ Products stream started")
print(f"  Query ID: {products_query.id}")
print(f"  Target: {bronze_products_table}")
print(f"  Checkpoint: {products_checkpoint}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Monitor Streaming Queries

# COMMAND ----------

print("\n" + "="*70)
print("ACTIVE STREAMING QUERIES")
print("="*70)

for stream in spark.streams.active:
    print(f"\nQuery ID: {stream.id}")
    print(f"  Name: {stream.name if stream.name else 'unnamed'}")
    print(f"  Status: {stream.status['message']}")
    print(f"  Is Active: {stream.isActive}")
    
    if stream.recentProgress:
        latest = stream.recentProgress[-1]
        print(f"  Recent Progress:")
        print(f"    - Batch: {latest.get('batchId', 'N/A')}")
        print(f"    - Input Rows: {latest.get('numInputRows', 0)}")
        print(f"    - Processing Rate: {latest.get('processedRowsPerSecond', 0):.2f} rows/sec")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Wait and Monitor (Run for 30 seconds)

# COMMAND ----------

import time

print("\n" + "="*70)
print("MONITORING STREAMS FOR 30 SECONDS")
print("="*70)

for i in range(6):
    time.sleep(5)
    
    # Check orders table
    try:
        orders_count = spark.table(bronze_orders_table).count()
        print(f"\n⏱️  {(i+1)*5}s - Orders: {orders_count} records")
    except:
        print(f"\n⏱️  {(i+1)*5}s - Orders table not created yet")
    
    # Check products table
    try:
        products_count = spark.table(bronze_products_table).count()
        print(f"    Products: {products_count} records")
    except:
        print(f"    Products table not created yet")

print("\n✓ Monitoring complete")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Verify Bronze Tables

# COMMAND ----------

print("\n" + "="*70)
print("BRONZE LAYER VERIFICATION")
print("="*70)

# Orders table
try:
    orders_df = spark.table(bronze_orders_table)
    orders_count = orders_df.count()
    
    print(f"\n📊 Orders Table: {bronze_orders_table}")
    print(f"   Total Records: {orders_count}")
    
    if orders_count > 0:
        print("\n   Latest 10 orders:")
        display(orders_df.orderBy(desc("bronze_timestamp")).limit(10))
        
        print("\n   Orders by Location:")
        display(
            orders_df.groupBy("location")
            .agg(
                count("*").alias("order_count"),
                sum("total_amount").alias("total_revenue")
            )
            .orderBy(desc("order_count"))
        )
except Exception as e:
    print(f"\n⚠️  Orders table not available: {e}")

# COMMAND ----------

# Products table
try:
    products_df = spark.table(bronze_products_table)
    products_count = products_df.count()
    
    print(f"\n📊 Products Table: {bronze_products_table}")
    print(f"   Total Records: {products_count}")
    
    if products_count > 0:
        print("\n   All products:")
        display(products_df.orderBy("product_id"))
        
        print("\n   Products by Category:")
        display(
            products_df.groupBy("category", "brand")
            .count()
            .orderBy("category", "brand")
        )
except Exception as e:
    print(f"\n⚠️  Products table not available: {e}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stream Management

# COMMAND ----------

# Option 1: Keep streams running
print("\n💡 Streams are running continuously")
print("   They will process new events as they arrive in Event Hub")
print("\n   To stop streams, run the next cell")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stop All Streams (Run when done)

# COMMAND ----------

# Uncomment to stop all streams
"""
print("=" * 70)
print("STOPPING ALL STREAMING QUERIES")
print("=" * 70)

for stream in spark.streams.active:
    print(f"\nStopping: {stream.id}")
    stream.stop()
    print(f"  ✓ Stopped")

print("\n✓ All streams stopped")
print("\nNote: You can restart streams by re-running the stream cells above")
"""

# COMMAND ----------

# MAGIC %md
# MAGIC ## Summary

# COMMAND ----------

print("\n" + "="*70)
print("✓ BRONZE LAYER STREAMING SETUP COMPLETE")
print("="*70)

try:
    orders_count = spark.table(bronze_orders_table).count()
    products_count = spark.table(bronze_products_table).count()
    
    print(f"\nCurrent State:")
    print(f"  Orders: {orders_count} records")
    print(f"  Products: {products_count} records")
    print(f"\nStreaming Status:")
    print(f"  Active Queries: {len(spark.streams.active)}")
    
    for stream in spark.streams.active:
        print(f"    - {stream.id}: {stream.status['message']}")
    
    print(f"\nData Flow:")
    print(f"  Event Hub ({eh_name})")
    print(f"    ↓ Kafka Streaming")
    print(f"  Parsed Events")
    print(f"    ↓ Split & Transform")
    print(f"  Bronze Tables")
    print(f"    • Orders (with product_id FK)")
    print(f"    • Products (deduplicated)")
    
    print("\n" + "="*70)
    print("Next Steps:")
    print("  1. Monitor streams using 'Monitor Streaming Queries' cell")
    print("  2. Run Silver Layer notebook to join Orders with Products")
    print("  3. Stop streams when done using 'Stop All Streams' cell")
    print("="*70)
    
except Exception as e:
    print(f"\n⚠️  Tables not created yet. Wait for streams to process data.")

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Bronze Layer - Orders Stream

# COMMAND ----------

# MAGIC %md
# MAGIC ## Configuration

# COMMAND ----------

# Event Hub Configuration


print("="*70)
print("STREAMING CONFIGURATION - ORDERS")
print("="*70)
print(f"Event Hub: {eh_name}")
print(f"Orders Table: {bronze_orders_table}")
print(f"Checkpoint: {orders_checkpoint}")
print("="*70)

# COMMAND ----------

from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# COMMAND ----------

# MAGIC %md
# MAGIC ## Schema Definition

# COMMAND ----------

event_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("customer_name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("payment_method", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("discount_pct", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("base_price", DoubleType(), True),
    StructField("unit_price", DoubleType(), True),
    StructField("order_timestamp", StringType(), True)
])

# COMMAND ----------

# MAGIC %md
# MAGIC ## Read from Event Hub

# COMMAND ----------

# Read streaming data
raw_stream = (spark.readStream
    .format("kafka")
    .options(**KAFKA_OPTIONS)
    .load()
)

# Parse JSON payload
parsed_stream = (raw_stream
    .selectExpr("CAST(value AS STRING) as json_value", "timestamp", "offset", "partition")
    .withColumn("parsed_data", from_json(col("json_value"), event_schema))
    .select(
        col("parsed_data.*"),
        col("timestamp").alias("event_time"),
        col("offset").alias("kafka_offset"),
        col("partition").alias("partition_id")
    )
    .withColumn("bronze_timestamp", current_timestamp())
    .withColumn("order_timestamp_parsed", to_timestamp(col("order_timestamp")))
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Write Orders Stream

# COMMAND ----------

# Select orders columns
orders_stream = parsed_stream.select(
    col("order_id"),
    col("customer_id"),
    col("customer_name"),
    col("location"),
    col("product_id"),
    col("order_status"),
    col("payment_method"),
    col("quantity"),
    col("discount_pct"),
    col("total_amount"),
    col("order_timestamp"),
    col("order_timestamp_parsed"),
    col("event_time"),
    col("kafka_offset"),
    col("partition_id"),
    col("bronze_timestamp")
)

# Write to Delta table
orders_query = (orders_stream
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", orders_checkpoint)
    .option("mergeSchema", "true")
    .trigger(processingTime="10 seconds")
    # .trigger(once=True)  # For testing
    .toTable(bronze_orders_table)
)

print(f"✓ Orders stream started: {orders_query.id}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Monitor Stream

# COMMAND ----------

for stream in spark.streams.active:
    print(f"Query: {stream.id}")
    print(f"Status: {stream.status['message']}")
    if stream.recentProgress:
        latest = stream.recentProgress[-1]
        print(f"Batch: {latest.get('batchId', 'N/A')}")
        print(f"Rows: {latest.get('numInputRows', 0)}")
        print(f"Rate: {latest.get('processedRowsPerSecond', 0):.2f} rows/sec")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Verify Orders Table

# COMMAND ----------

orders_df = spark.table(bronze_orders_table)
print(f"Total Orders: {orders_df.count()}")

display(orders_df.orderBy(desc("bronze_timestamp")).limit(10))

# Orders by location
display(
    orders_df.groupBy("location")
    .agg(
        count("*").alias("order_count"),
        sum("total_amount").alias("total_revenue")
    )
    .orderBy(desc("order_count"))
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stop Stream

# COMMAND ----------

# Uncomment to stop
# for stream in spark.streams.active:
#     stream.stop()
#     print(f"Stopped: {stream.id}")

In [0]:

# COMMAND ----------

from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# COMMAND ----------

# MAGIC %md
# MAGIC ## Schema Definition

# COMMAND ----------

event_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("customer_name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("payment_method", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("discount_pct", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("base_price", DoubleType(), True),
    StructField("unit_price", DoubleType(), True),
    StructField("order_timestamp", StringType(), True)
])

# COMMAND ----------

# MAGIC %md
# MAGIC ## Read from Event Hub

# COMMAND ----------

# Read streaming data
raw_stream = (spark.readStream
    .format("kafka")
    .options(**KAFKA_OPTIONS)
    .load()
)

# Parse JSON payload
parsed_stream = (raw_stream
    .selectExpr("CAST(value AS STRING) as json_value", "timestamp", "offset", "partition")
    .withColumn("parsed_data", from_json(col("json_value"), event_schema))
    .select(
        col("parsed_data.*"),
        col("timestamp").alias("event_time"),
        col("offset").alias("kafka_offset"),
        col("partition").alias("partition_id")
    )
    .withColumn("bronze_timestamp", current_timestamp())
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Upsert Function

# COMMAND ----------

def upsert_products(batch_df, batch_id):
    """Upsert products to avoid duplicates (SCD Type 1)"""
    if batch_df.count() == 0:
        return
    
    batch_df.createOrReplaceTempView("products_batch")
    
    merge_query = f"""
    MERGE INTO {bronze_products_table} target
    USING products_batch source
    ON target.product_id = source.product_id
    WHEN MATCHED THEN
        UPDATE SET
            target.product_name = source.product_name,
            target.category = source.category,
            target.brand = source.brand,
            target.base_price = source.base_price,
            target.unit_price = source.unit_price,
            target.bronze_timestamp = source.bronze_timestamp
    WHEN NOT MATCHED THEN
        INSERT (product_id, product_name, category, brand, base_price, unit_price, bronze_timestamp)
        VALUES (source.product_id, source.product_name, source.category, source.brand, 
                source.base_price, source.unit_price, source.bronze_timestamp)
    """
    
    try:
        spark.sql(merge_query)
        print(f"Batch {batch_id}: Upserted {batch_df.count()} products")
    except Exception as e:
        if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
            batch_df.write.format("delta").mode("append").saveAsTable(bronze_products_table)
            print(f"Batch {batch_id}: Created table, inserted {batch_df.count()} products")
        else:
            raise e

# COMMAND ----------

# MAGIC %md
# MAGIC ## Write Products Stream

# COMMAND ----------

# Select and deduplicate products
products_stream = (parsed_stream
    .select(
        col("product_id"),
        col("product_name"),
        col("category"),
        col("brand"),
        col("base_price"),
        col("unit_price"),
        col("bronze_timestamp")
    )
    .dropDuplicates(["product_id"])
)

# Write with upsert logic
products_query = (products_stream
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", products_checkpoint)
    .option("mergeSchema", "true")
    .trigger(processingTime="10 seconds")
    # .trigger(once=True)  # For testing
    .foreachBatch(upsert_products)
    .start()
)

print(f"✓ Products stream started: {products_query.id}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Monitor Stream

# COMMAND ----------

for stream in spark.streams.active:
    print(f"Query: {stream.id}")
    print(f"Status: {stream.status['message']}")
    if stream.recentProgress:
        latest = stream.recentProgress[-1]
        print(f"Batch: {latest.get('batchId', 'N/A')}")
        print(f"Rows: {latest.get('numInputRows', 0)}")
        print(f"Rate: {latest.get('processedRowsPerSecond', 0):.2f} rows/sec")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Verify Products Table

# COMMAND ----------

products_df = spark.table(bronze_products_table)
print(f"Total Unique Products: {products_df.count()}")

display(products_df.orderBy("product_id"))

# Products by category
display(
    products_df.groupBy("category", "brand")
    .count()
    .orderBy("category", "brand")
)

# Verify no duplicates
duplicate_check = (products_df
    .groupBy("product_id")
    .count()
    .filter(col("count") > 1)
)
print(f"Duplicate product_ids: {duplicate_check.count()}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Stop Stream

# COMMAND ----------

# Uncomment to stop
# for stream in spark.streams.active:
#     stream.stop()
#     print(f"Stopped: {stream.id}")

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Quick Event Hub Test

# COMMAND ----------

from pyspark.sql.functions import *

# Event Hub Configuration
eh_namespace = "evhns-natraining.servicebus.windows.net"
eh_name = "evh-natraining-biju"
keyvault_scope = "dbx-ss-kv-natraining-2"
secret_name = "evh-natraining-read-write"
shared_access_key_name = "SharedAccessKeyToSendAndListen"

sas_key = dbutils.secrets.get(scope=keyvault_scope, key=secret_name)

# Kafka options
KAFKA_OPTIONS = {
    "kafka.bootstrap.servers": f"{eh_namespace}:9093",
    "subscribe": eh_name,
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://{eh_namespace}/;SharedAccessKeyName={shared_access_key_name};SharedAccessKey={sas_key};";',
    "failOnDataLoss": "false",
    "startingOffsets": "earliest"
}

print(f"Connecting to Event Hub: {eh_name}")

# COMMAND ----------

# Read events
events = spark.read.format("kafka").options(**KAFKA_OPTIONS).load()

print(f"✓ Connected! Total events: {events.count()}")

# COMMAND ----------

# View raw events
display(events.limit(10))

# COMMAND ----------

# Parse JSON payload
parsed = (events
    .selectExpr("CAST(value AS STRING) as json_value", "timestamp", "offset", "partition")
)

display(parsed.limit(10))

# COMMAND ----------

# Extract specific fields (adjust based on your JSON structure)
details = (parsed
    .select(
        col("timestamp"),
        col("offset"),
        get_json_object(col("json_value"), "$.order_id").alias("order_id"),
        get_json_object(col("json_value"), "$.customer_name").alias("customer_name"),
        get_json_object(col("json_value"), "$.product_name").alias("product_name"),
        get_json_object(col("json_value"), "$.location").alias("location"),
        get_json_object(col("json_value"), "$.total_amount").alias("total_amount")
    )
)

display(details.limit(20))

# COMMAND ----------

# Test streaming
stream = (spark.readStream
    .format("kafka")
    .options(**KAFKA_OPTIONS)
    .load()
)

query = (stream
    .selectExpr("CAST(value AS STRING) as json_value")
    .writeStream
    .format("console")
    .outputMode("append")
    .trigger(processingTime="5 seconds")
    .start()
)

print(f"Streaming query started: {query.id}")

# COMMAND ----------

# Stop stream
for s in spark.streams.active:
    s.stop()
print("✓ Streams stopped")